In [1]:
import pandas as pd
import numpy as np

#### Training set

In [2]:
train_df = pd.read_csv("./DATA/training.csv", nrows= 100)

In [3]:
imgs = train_df['Image'].str.split(" ", expand= True).to_numpy(dtype= float).reshape(-1, 96, 96)

In [4]:
imgs.shape

(100, 96, 96)

In [5]:
np.sum(np.isnan(imgs))

0

In [6]:
keypoints = train_df[train_df.columns[:30]].to_numpy(dtype= float)

In [7]:
keypoints.shape

(100, 30)

#### Test set

In [8]:
test_df = pd.read_csv("./DATA/test.csv", nrows= 10)

In [9]:
test_data = test_df['Image'].str.split(" ", expand= True).to_numpy().reshape(-1, 96, 96)

In [10]:
test_data.shape

(10, 96, 96)

#### simple keras

In [11]:
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Reshape, Conv2D, BatchNormalization
import keras.backend as K

2022-05-06 16:53:14.953173: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-06 16:53:14.953239: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [12]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [13]:
model = Sequential()
model.add(InputLayer((96, 96)))
model.add(Reshape((96, 96, 1)))
model.add(Conv2D(filters= 32, kernel_size= 3, padding= "same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters= 64, kernel_size= 3, padding= "same", activation="relu"))
model.add(BatchNormalization())
model.add(Reshape((-1, )))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(30, activation='relu'))

model.compile(loss= root_mean_squared_error, optimizer='adam', metrics=['accuracy'])

2022-05-06 16:53:17.783168: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-06 16:53:17.783242: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-06 16:53:17.783510: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LoG): /proc/driver/nvidia/version does not exist
2022-05-06 16:53:17.784206: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 16:53:17.984541: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2415919104 exceeds 10% 

In [14]:
print(imgs.shape)
print(keypoints.shape)

(100, 96, 96)
(100, 30)


In [16]:
model.fit(imgs, keypoints, epochs= 100, batch_size= 4, validation_split= 0.8)

Epoch 1/100
2/5 [===========>..................] - ETA: 28s - loss: 49.4996 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_data)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)